In [1]:
# PTT 專用爬蟲
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
from time import sleep
import csv
import random

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
}
KEYWORDS = ["保險", "國泰", "國泰人壽", "保單", "理賠", "詐騙", "推薦", "壽險", "產險"]
PTT_BOARDS = ["Insurance", "WomenTalk", "Stock", "Lifeismoney", "Gossiping"]
MAX_PER_KEYWORD = 5000


def save_to_csv(filename, rows):
    with open(filename, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["來源", "標題", "連結", "內容"])
        writer.writerows(rows)


def crawl_ptt(keyword):
    base_url = "https://www.ptt.cc"
    rows = []
    for board in PTT_BOARDS:
        search_url = f"https://www.ptt.cc/bbs/{board}/search?q={quote(keyword)}"
        try:
            res = requests.get(search_url, headers=HEADERS)
            soup = BeautifulSoup(res.text, "html.parser")
            links = soup.select(".title a")
            for a in links[:MAX_PER_KEYWORD // len(PTT_BOARDS)]:
                url = base_url + a['href']
                title = a.text.strip()
                post = requests.get(url, headers=HEADERS)
                post_soup = BeautifulSoup(post.text, "html.parser")
                content = post_soup.select_one("#main-content").text
                rows.append([f"PTT-{board}", title, url, content])
                pushes = post_soup.select(".push")
                for p in pushes:
                    msg = p.text.strip()
                    rows.append([f"PTT-{board}-回文", title, url, msg])
                sleep(random.uniform(0.5, 1.2))
        except:
            continue
    return rows


all_data = []
for kw in KEYWORDS:
    all_data += crawl_ptt(kw)

save_to_csv("ptt_語料.csv", all_data)
print(f"PTT 完成，共 {len(all_data)} 筆")


PTT 完成，共 36023 筆
